In [ ]:
import requests
from urllib import parse

url = "https://comic.naver.com/webtoon/detail?titleId=570503&no=392&weekday=thu"

# make valid header
parsed_url = parse.urlparse(url)
parsed_url_query = dict(parse.parse_qsl(parsed_url.query))
for key in list(parsed_url_query.keys()): # remove weekday value
    if key not in ("titleId", "no"):
        parsed_url_query.pop(key)
comment_url = parse.urlunparse(parsed_url._replace(path="/comment/comment", query=parse.urlencode(parsed_url_query)))
header = {"referer":comment_url,
 "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/97.0.4692.99 Safari/537.36"}

# get api url using query of init api url
init_api_url = "https://apis.naver.com/commentBox/cbox/web_naver_list_jsonp.json?ticket=comic&pool=cbox3&lang=ko&objectId=183559_517&pageSize=100&page=1"
parsed_init_api_url = parse.urlparse(init_api_url)
parsed_init_api_url_query = dict(parse.parse_qsl(parsed_init_api_url.query))
new_objectId = "_".join(parsed_url_query.values())
parsed_init_api_url_query["objectId"] = new_objectId # change only objectId query value
api_url = parse.urlunparse(parsed_init_api_url._replace(query=parse.urlencode(parsed_init_api_url_query)))

In [ ]:
import re
import json

# removing useless prefix and subfix
removeElse = re.compile("^[^{]+|[^}]+$")
api_json = json.loads(removeElse.sub("", requests.get(api_url, headers=header).text))
comment_list = None
if api_json["success"]:
    comment_list = api_json["result"]["commentList"]
len(comment_list)

In [ ]:
total_text = ""

text_cleaning = re.compile("[^가-힣 ]")

for i in comment_list:
    print("---------------------------------------")
    print(i["userIdNo"], i["regTime"], i["sympathyCount"], i["antipathyCount"])
    print(i["contents"])
    total_text += " " + text_cleaning.sub(" ", i["contents"])
    if i["modTime"] != i["regTime"]:
        print(i)
        break

In [ ]:
total_text

In [ ]:
from konlpy.tag import Okt
from collections import Counter

noun_counter = Counter(Okt().nouns(total_text))

In [ ]:
noun_counter

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

wordcloud = WordCloud(font_path='NanumGothic', background_color='white', colormap="cool").fit_words(noun_counter)
plt.imshow(wordcloud)
plt.axis("off")
# wordcloud.to_image().show()
# wordcloud.to_file('test.jpg')

In [ ]:
total_text